In [21]:
import torch as t
import torch.nn as nn 
from torch.nn import functional as F 

device = 'cuda' if t.cuda.is_available() else 'cpu'
print(device)

block_size = 8
batch_size =4

cpu


In [8]:
with open("wizard_of_oz.text","r",encoding="utf-8") as f:
    text = f.read()
    print(text[:200])

﻿DOROTHY AND THE WIZARD IN OZ

  BY

  L. FRANK BAUM

  AUTHOR OF THE WIZARD OF OZ, THE LAND OF OZ, OZMA OF OZ, ETC.

  ILLUSTRATED BY JOHN R. NEILL

  BOOKS OF WONDER WILLIAM MORROW & CO., INC. NEW Y


In [9]:
##Character Level Encoding / Decoding

chars = sorted(set(text))
print(chars)
print(len(chars))

### Encoding and Decoding
str_to_int = {ch:i for i,ch in enumerate(chars)}
encode = lambda s : [str_to_int[c] for c in s]
str = "Hello"
encoding = encode(str)
print('encoding ',str, ' -> ',encoding )

int_to_str = {i:ch for i,ch in enumerate(chars)}
decode = lambda list: ''.join([int_to_str[i] for i in list])
print('decoding ',encoding, ' -> ', decode(encode("Hello")))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81
encoding  Hello  ->  [32, 58, 65, 65, 68]
decoding  [32, 58, 65, 65, 68]  ->  Hello


In [22]:
# Big Boy Stuff

data = t.tensor(encode(text), dtype=torch.long)
print(data[:200])

# Split the data into training (first 80%) and validation (remaining 20%) datasets.
n = int(0.8 * len(data))
train_data = data[:n]
val_data = data[n:]


# Simple Biagram 

x = train_data[:block_size]
y = train_data[1:block_size+1]

for i in range(block_size): 
    context = x[:i+1]
    target = y[i]
    print('when context is', context, ' target is ', target)



# Get batch function

def get_batch(split):
    """
    Generate a batch of data for training or validation.

    Parameters:
    split (str): A string that can be 'train' or 'val' indicating whether
                 to fetch training or validation data.

    Returns:
    Tuple[Tensor, Tensor]: A tuple of two tensors, inputs (x) and targets (y).
                           Each tensor contains a batch of sequences.
    """
    # Select the appropriate data split based on the 'split' argument.
    data = train_data if split == "train" else val_data

    # Randomly select starting indices for sequences in the batch. 
    # Ensures selected sequences will fit within the remaining data.
    ix = t.randint(len(data) - block_size, (batch_size,))

    # Create the input batch by selecting sequences starting from indices in ix.
    x = t.stack([data[i:i + block_size] for i in ix])

    # Create the target batch, offset by one element from the input batch,
    # to align with the bigram prediction task.
    y = t.stack([data[i + 1:i + block_size + 1] for i in ix])

    return x, y


# Fetch and print a batch of training data

# Get a batch of training data.
x, y = get_batch("train")

# Print the input sequences of the batch.
print(f"inputs: {x}")

# Print the corresponding target sequences of the batch. 
print(f"targets: {y}")


tensor([80, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,  0,
         0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,  0,
         1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47, 33,
        50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36, 25,
        38, 28,  1, 39, 30,  1, 39, 50,  9,  1, 39, 50, 37, 25,  1, 39, 30,  1,
        39, 50,  9,  1, 29, 44, 27, 11,  0,  0,  1,  1, 33, 36, 36, 45, 43, 44,
        42, 25, 44, 29, 28,  1, 26, 49,  1, 34, 39, 32, 38,  1, 42, 11,  1, 38,
        29, 33, 36, 36,  0,  0,  1,  1, 26, 39, 39, 35, 43,  1, 39, 30,  1, 47,
        39, 38, 28, 29, 42,  1, 47, 33, 36, 36, 33, 25, 37,  1, 37, 39, 42, 42,
        39, 47,  1,  4,  1, 27, 39, 11,  9,  1, 33, 38, 27, 11,  1, 38, 29, 47,
         1, 49])
when context is tensor([80])  target is  tensor(28)
when context is tensor([80, 28])  target is  tensor

In [27]:
# Create a Neural Net

class BiagramLanguageModel(nn.Module):
    def _init_(self,vocab_size):
        super()._init_()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self,index,targets):
        logits = self.token_embedding_table(index)

        if targets == None:
            loss = None
        else:     
            B , T , C  = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy
        
        return logits, loss


     def generate(self, index, max_new_tokens):


In [ ]:
## Neural Net 